# Neural Knapsack

### What is the knapsack problem?

This is how my lovely computer algorithm professor explained the knapsack problem to me:

Imagine a thief on a robbery, there are a couple of items to choose from where each item has its weight and value. The thief has a knapsack with a determined capacity and the thief can't carry items more than the capacity of his(or her) knapsack(the sum of chosen items' weights must be less than or equal to the knapsack's capacity). Our goal is to maximize the thief's profit by choosing the right items.

But according to wikipedia [1]:

"Given a set of items, each with a weight and a value, determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible. It derives its name from the problem faced by someone who is constrained by a fixed-size knapsack and must fill it with the most valuable items".


### What are we trying to do?

We want to somehow manage to solve the knapsack problem using the neural networks only.

### Imports

Before we begin, we need to import the necessary libraries.

In [14]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Activation, Lambda, Input, Concatenate, Multiply
from keras.metrics import binary_accuracy
from keras.losses import binary_crossentropy
import keras.backend as K

### Problem creation

At first, we need to create some knapsack problems. We use random integers for items' weights, items' prices and the capacity of the knapsack. Along with each problem, we use brute force to find the optimum answer to the problem as the number of items in each problem is small enough.

In [15]:
def test_knapsack(x_weights, x_prices, x_capacity, picks):
    total_price = np.dot(x_prices, picks)
    total_weight = np.dot(x_weights, picks)
    return total_price, max(total_weight - x_capacity, 0)


def brute_force_knapsack(x_weights, x_prices, x_capacity):
    picks_space = 2 ** x_weights.shape[0]
    best_price = 0
    best_picks = None
    for p in range(picks_space):
        picks = np.zeros((x_weights.shape[0]))
        for i, c in enumerate("{0:b}".format(p)[::-1]):
            picks[i] = c
        price, violation = test_knapsack(x_weights, x_prices, x_capacity, picks)
        if violation == 0:
            if price > best_price:
                best_price = price
                best_picks = picks
    return best_price, best_picks

def create_knapsack(item_count=5):
    x_weights = np.random.randint(1, 15, item_count)
    x_prices = np.random.randint(1, 10, item_count)
    x_capacity = np.random.randint(15, 50)
    _, y_best_picks = brute_force_knapsack(x_weights, x_prices, x_capacity)
    return x_weights, x_prices, x_capacity, y_best_picks

Each of the knapsack problems is shown like this:

In [16]:
x_weights, x_prices, x_capacity, y_best_picks = create_knapsack()
print("Weights:", x_weights)
print("Prices:", x_prices)
print("Capacity:", x_capacity)
print("Best picks:", y_best_picks)

Weights: [6 4 7 9 1]
Prices: [4 3 8 4 3]
Capacity: 23
Best picks: [1. 0. 1. 1. 1.]


### Metrics

Beside binary classification accuracy(which compares the models' picks and optimum answer), knapsack problem needs a couple of other metrics to be evaluated with. The first metric I want to present is called "overpricing" as its name suggests, it shows the average difference between the price that the neural net has picked and the price of the optimum answer. It is called overpricing dude to the fact that the neural net most of the times becomes greedy and picks more than the optimum answer. Overpricing is defined in Keras framework as follow:

In [17]:
def metric_overprice(input_prices):
    def overpricing(y_true, y_pred):
        y_pred = K.round(y_pred)
        return K.mean(K.batch_dot(y_pred, input_prices, 1) - K.batch_dot(y_true, input_prices, 1))

    return overpricing

Another metric that is important to our cause is the amount of space that is used over the amount of actual capacity of the knapsack. The metric "space violation" is the difference between picked items required space and knapsacks capacity for positive values(when NN picked more than the capacity of knapsack) and zero for negative ones. Space violation is defined in Keras framework as follow:

In [18]:
def metric_space_violation(input_weights, input_capacity):
    def space_violation(y_true, y_pred):
        y_pred = K.round(y_pred)
        return K.mean(K.maximum(K.batch_dot(y_pred, input_weights, 1) - input_capacity, 0))

    return space_violation

The last metric, I choose for knapsack problem is called "pick count". This variable shows the number of items that the neural net picks more/less than the optimum solution. The metric could be defined in Keras like:

In [19]:
def metric_pick_count():
    def pick_count(y_true, y_pred):
        y_pred = K.round(y_pred)
        return K.mean(K.sum(y_pred, -1) - K.sum(y_true, -1))

    return pick_count

## Solutions

### Dataset

First, we create a dataset consist of ten thousand of knapsack samples for training and two hundred for testing.

In [20]:
def create_knapsack_dataset(count, item_count=5):
    x = [[], [], []]
    y = [[]]
    for _ in range(count):
        p = create_knapsack(item_count)
        x[0].append(p[0])
        x[1].append(p[1])
        x[2].append(p[2])
        y[0].append(p[3])
    return x, y
train_x, train_y = create_knapsack_dataset(10000)
test_x, test_y = create_knapsack_dataset(200)

C:\Users\Thumbby\AppData\Local\Temp\ipykernel_8036\2449799711.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_x = np.asarray(train_x)
C:\Users\Thumbby\AppData\Local\Temp\ipykernel_8036\2449799711.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_x = np.asarray(test_x)


Now we create a function to train our models for the sake of simplicity. The function is defind like:

In [21]:
def train_knapsack(model):
    from keras.callbacks import ModelCheckpoint
    import os
    if os.path.exists("best_model.h5"): os.remove("best_model.h5")
    model.fit(train_x, train_y, epochs=96, verbose=0, callbacks=[ModelCheckpoint("best_model.h5", monitor="loss", save_best_only=True, save_weights_only=True)])
    model.load_weights("best_model.h5")
    train_results = model.evaluate(train_x, train_y, 64, 0)
    test_results = model.evaluate(test_x, test_y, 64, 0)
    print("Model results(Train/Test):")
    print(f"Loss:               {train_results[0]:.2f} / {test_results[0]:.2f}")
    print(f"Binary accuracy:    {train_results[1]:.2f} / {test_results[1]:.2f}")
    print(f"Space violation:    {train_results[2]:.2f} / {test_results[2]:.2f}")
    print(f"Overpricing:        {train_results[3]:.2f} / {test_results[3]:.2f}")
    print(f"Pick count:         {train_results[4]:.2f} / {test_results[4]:.2f}")

The function trains a neural knapsack model and prints its loss and metrics that we have discussed before.

### Supervised continues solution

The first solution that pops up to mind is to input three variables and expect the model to learn the picks(selected items) as the output and train the model using cross entropy loss. Since the output of the model is not discrete(only 0 and 1), we name it the supervised continuous model.

In [22]:
def supervised_continues_knapsack(item_count=5):
    input_weights = Input((item_count,))
    input_prices = Input((item_count,))
    input_capacity = Input((1,))
    inputs_concat = Concatenate()([input_weights, input_prices, input_capacity])
    picks = Dense(item_count, use_bias=False, activation="sigmoid")(inputs_concat)
    model = Model(inputs=[input_weights, input_prices, input_capacity], outputs=[picks])
    model.compile("sgd",
                  binary_crossentropy,
                  metrics=[binary_accuracy, metric_space_violation(input_weights, input_capacity),
                           metric_overprice(input_prices), metric_pick_count()])
    return model
model = supervised_continues_knapsack()
train_knapsack(model)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

As it is expected, we can get slightly better accuracy by adding a hidden layer.

In [ ]:
def supervised_continues_knapsack_one_hidden(item_count=5):
    input_weights = Input((item_count,))
    input_prices = Input((item_count,))
    input_capacity = Input((1,))
    inputs_concat = Concatenate()([input_weights, input_prices, input_capacity])
    picks = Dense(item_count * 10, use_bias=False, activation="sigmoid")(inputs_concat)
    picks = Dense(item_count, use_bias=False, activation="sigmoid")(picks)
    model = Model(inputs=[input_weights, input_prices, input_capacity], outputs=[picks])
    model.compile("sgd",
                  binary_crossentropy,
                  metrics=[binary_accuracy, metric_space_violation(input_weights, input_capacity),
                           metric_overprice(input_prices), metric_pick_count()])
    return model
model = supervised_continues_knapsack_one_hidden()
train_knapsack(model)

Model results(Train/Test):
Loss:               0.23 / 0.23
Binary accuracy:    0.90 / 0.90
Space violation:    1.11 / 1.50
Overpricing:        0.70 / 0.85
Pick count:         0.12 / 0.15


It can be inferred that the neural network is able to learn and generalize the knapsack problem. But there are a couple of problems with the supervised continuous approach:
* The output of the model is continuous however our picks must be discrete.
* The optimum answer is mandatory for training though it takes a reasonable amount of time to compute each.
* We can't trade off between space violation and overpricing.

Through the rest of this article, we try to address these problems.

### Supervised discrete solution

We can make the output of the model discrete by passing it through a round function. The problem is raised when we attempt to train the model as the gradient is not able to pass through the round function hence our model will not be trainable. So, we need a continues function which outputs only ones and zeros! We can make some compromises due to the fact that the mentioned function is not available in math yet. We can make the values so close to zero or one that the final result won't differ that much from actual one or zero. As described by [2], we can achieve the values close to -1, 0 or 1 by multiplying the output of two parallel fully-connected layers with $tanh$ and $sigmoid$ activations(more information could be obtained from the original paper). As we don't want -1 to be a possible value for the output, we use $$f(x)=(tanh(W_1 \cdot x) + \sigma(W_2 \cdot x))^2$$where $W_1$ and $W_2$ are weight matrices.

In [ ]:
def supervised_discrete_knapsack(item_count=5):
    input_weights = Input((item_count,))
    input_prices = Input((item_count,))
    input_capacity = Input((1,))
    inputs_concat = Concatenate()([input_weights, input_prices, input_capacity])
    concat_tanh = Dense(item_count, use_bias=False, activation="tanh")(inputs_concat)
    concat_sigmoid = Dense(item_count, use_bias=False, activation="sigmoid")(inputs_concat)
    concat_multiply = Multiply()([concat_sigmoid, concat_tanh])
    picks = Multiply()([concat_multiply, concat_multiply])
    model = Model(inputs=[input_weights, input_prices, input_capacity], outputs=[picks])
    model.compile("sgd",
                  binary_crossentropy,
                  metrics=[binary_accuracy, metric_space_violation(input_weights, input_capacity),
                           metric_overprice(input_prices), metric_pick_count()])
    return model

model = supervised_discrete_knapsack()
train_knapsack(model)

UnboundLocalError: local variable 'train_x' referenced before assignment

We tackled the first problem by introducing a new model which outputs semi-discrete values. In order to handle the other problems, we need to introduce a new way of training and loss function.

### Unsupervised discrete solution

As we want to train our model without computing the optimum answer, we need a new loss function to train our neural knapsack models. The new loss function should not be in need of target outputs.
So we subsequently define the $w$, $p$, $c$ and $o$ as items' weights, items' prices, knapsack's capacity and model's output(picks). The first need of the knapsack problem is maximizing the picked items total price. The total price may be calculated via $$TotalPrice=p \cdot o$$

Meanwhile we want our picks not to surpass the knapsack's capacity. This value may be formulated like$$SpaceViolation=\max{(w \cdot o - c,\ 0)}$$


Finally, we want to maximize the $TotalPrice$ while minimizing $SpaceViolation$. This could be formulated like:$$J=-TotalPrice + SpaceViolation$$


In order to be familiar with popular loss, we set our first term as the optimization term and the second one as the regularization term. With that in mind, we can add the $\lambda$ coefficient for the regularization term. The final loss formula is $$J=-TotalPrice + \lambda SpaceViolation$$


We can implement the new loss in Keras like this:

In [ ]:
def knapsack_loss(input_weights, input_prices, input_capacity, cvc=1):
    def loss(y_true, y_pred):
        picks = y_pred
        return (-1 * K.batch_dot(picks, input_prices, 1)) + cvc * K.maximum(
            K.batch_dot(picks, input_weights, 1) - input_capacity, 0)
    return loss

Using this loss function, we hope to train the neural network in an unsupervised fashion.

**Note 1:** The $\lambda$ parameter is named *cvc* in the Keras implementation of knapsack loss.

**Note 2:** Although we received the expected output(y_true),  we do not use it for computing the loss function. We can't either omit the argument as it is a Keras constraint to receive both predicted and expected outputs in loss functions.

In [ ]:
def unsupervised_discrete_knapsack(item_count=5):
    input_weights = Input((item_count,))
    input_prices = Input((item_count,))
    input_capacity = Input((1,))
    inputs_concat = Concatenate()([input_weights, input_prices, input_capacity])
    concat_tanh = Dense(item_count, use_bias=False, activation="tanh")(inputs_concat)
    concat_sigmoid = Dense(item_count, use_bias=False, activation="sigmoid")(inputs_concat)
    concat_multiply = Multiply()([concat_sigmoid, concat_tanh])
    picks = Multiply()([concat_multiply, concat_multiply])
    model = Model(inputs=[input_weights, input_prices, input_capacity], outputs=[picks])
    model.compile("sgd",
                  knapsack_loss(input_weights, input_prices, input_capacity, 1),
                  metrics=[binary_accuracy, metric_space_violation(input_weights, input_capacity),
                           metric_overprice(input_prices), metric_pick_count()])
    return model
model = unsupervised_discrete_knapsack()
train_knapsack(model)

Model results(Train/Test):
Loss:               -20.24 / -20.04
Binary accuracy:    0.84 / 0.85
Space violation:    2.44 / 2.48
Overpricing:        1.04 / 1.16
Pick count:         0.34 / 0.36


We experienced a 3 percent loss in binary accuracy method in comparison to the supervised model. We can address this problem by adding a hidden layer.

In [ ]:
def unsupervised_discrete_knapsack_one_hidden(item_count=5):
    input_weights = Input((item_count,))
    input_prices = Input((item_count,))
    input_capacity = Input((1,))
    inputs_concat = Concatenate()([input_weights, input_prices, input_capacity])
    inputs_concat = Dense(item_count * 10, activation="relu")(inputs_concat)
    concat_tanh = Dense(item_count, use_bias=False, activation="tanh")(inputs_concat)
    concat_sigmoid = Dense(item_count, use_bias=False, activation="sigmoid")(inputs_concat)
    concat_multiply = Multiply()([concat_sigmoid, concat_tanh])
    picks = Multiply()([concat_multiply, concat_multiply])
    model = Model(inputs=[input_weights, input_prices, input_capacity], outputs=[picks])
    model.compile("sgd",
                  knapsack_loss(input_weights, input_prices, input_capacity),
                  metrics=[binary_accuracy, metric_space_violation(input_weights, input_capacity),
                           metric_overprice(input_prices), metric_pick_count()])
    return model
model = unsupervised_discrete_knapsack_one_hidden()
train_knapsack(model)

Model results(Train/Test):
Loss:               -21.79 / -21.49
Binary accuracy:    0.90 / 0.89
Space violation:    1.51 / 1.77
Overpricing:        1.24 / 1.41
Pick count:         0.28 / 0.30


By increasing the *cvc*($\lambda$) parameter of the lost function, we can force the neural network not to violate the space of the knapsack.

In [ ]:
def unsupervised_discrete_knapsack_one_hidden(item_count=5):
    input_weights = Input((item_count,))
    input_prices = Input((item_count,))
    input_capacity = Input((1,))
    inputs_concat = Concatenate()([input_weights, input_prices, input_capacity])
    inputs_concat = Dense(item_count * 10, activation="relu")(inputs_concat)
    concat_tanh = Dense(item_count, use_bias=False, activation="tanh")(inputs_concat)
    concat_sigmoid = Dense(item_count, use_bias=False, activation="sigmoid")(inputs_concat)
    concat_multiply = Multiply()([concat_sigmoid, concat_tanh])
    picks = Multiply()([concat_multiply, concat_multiply])
    model = Model(inputs=[input_weights, input_prices, input_capacity], outputs=[picks])
    model.compile("sgd",
                  knapsack_loss(input_weights, input_prices, input_capacity, 5),
                  metrics=[binary_accuracy, metric_space_violation(input_weights, input_capacity),
                           metric_overprice(input_prices), metric_pick_count()])
    return model
model = unsupervised_discrete_knapsack_one_hidden()
train_knapsack(model)

Model results(Train/Test):
Loss:               -20.02 / -19.59
Binary accuracy:    0.87 / 0.86
Space violation:    0.23 / 0.24
Overpricing:        -1.05 / -1.20
Pick count:         -0.22 / -0.23


As demonstrated in the above experience, the space violation is reduced by $88\%$ while losing only $3\%$ of binary accuracy.

## Conclusion

As discussed before, we solved the problem of knapsack using neural networks while the gradient can pass through the network. So our model can be trained end to end using any variation of the backpropagation algorithm.

This research can be used in recommender systems, especially the ones which can recommend multiple items while maintaining space usage (banner Ads could be a good example).

## References
[1] Knapsack problem - Wikipedia, [Wikipedia.com](https://en.wikipedia.org/wiki/Knapsack_problem), July 2019

[2] A. Trask, F. Hill, S. E. Reed, J. Rae, C. Dyer, and P. Blunsom, “Neural arithmetic logic units,” in Advances in Neural Information Processing Sys- tems, 2018, pp. 8035–8044. [Arxiv.org](https://arxiv.org/abs/1808.00508)